### Embeddinge skinuti s google drivea ispod te staviti u folder u kojemu se nalazi notebook.
### Embeddinzi su izračunati u drugom notebooku.
### Trening traje 4+ sati pa da se skrati proces.

### https://drive.google.com/drive/folders/1YFfS220nDpc59b7SxNp2e4omSWnemtSx

# TRENING MODELA

In [3]:
import numpy as np
import pandas as pd
import requests
import pickle
import tensorflow as tf


from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
from seqeval.metrics import recall_score

from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.layers.merge import add


from sklearn.model_selection import train_test_split
from elmoformanylangs import Embedder




def kreiraj_dataset():
   
   #Connllu file se skida s interneta te učitava tekst koji se potom obrađuje.
   #Funkcija na kraju vraća dataframe s istim stupcima koji su u conllu fajlu

    url = "https://raw.githubusercontent.com/reldi-data/hr500k/master/hr500k.conllu"
    r = requests.get(url, allow_redirects=True)
    open('hr500k.conllu', 'wb').write(r.content)

    tekst=[]
    for tek in open('hr500k.conllu',"r",encoding="utf8") .readlines():
            if tek.startswith("#") or tek.startswith("\n"):  
              pass
            else:
              tekst.append(tek.split("\t"))  


    tekst=pd.DataFrame(tekst)


    tekst=tekst.drop([6,8],axis=1)
    tekst=tekst.rename(columns={0:"Word index", 1:"Riječ",2:"Korijen",3:"UPOS",4:"XPOS",5:"FEATS",7:"DEPREL",9:"Ostalo"})
    NER_tagovi=[]

    for tag in tekst["Ostalo"]:
        ind=tag.find("NER=")
        if ind!=-1:
          NER_tagovi.append(str.upper(tag[ind+4:ind+9]))
        else:
          NER_tagovi.append("O")

    tekst["NER_tag"]=NER_tagovi
    recenice=[]
    broj_recenice=0
    

    for ind in tekst["Word index"]:
      if int(ind)==1:
        broj_recenice+=1
        recenice.append(broj_recenice)
      else:
        recenice.append(broj_recenice)
        

    tekst["Rečenica"]=recenice

    #tekst.to_csv('tekst.csv')

    return tekst


def predvidi_jedan(i):
  #Ova funkcija uzima instancu iz test skupa te računa vjerojatnost da token pripada nekom NER tagu. Tada uzme NER tag za koji je predviđena najveća vjerojatnost i spremi tu vrijednost(broj), te isto tako za ostale riječi u rečenici.       
  #U konačnici p je lista predviđenih NER tagova.
  #y_true su stvarni NER tagovi.
  #Pomoću riječnika indeksi_u_tagove (definiranog u kodu dolje) se dohvaćaju NER tagovi kao stringovi i list p koju funkcija vraća je lista NER stringova (za svaku riječ u rečenici).

  p = model.predict(np.array(X_te[i:i+batch_size]))[0]
  p = np.argmax(p, axis=-1)
  p=list(p)
    
     
  y_true=[]
  for i in y_te[i]:
   y_true.append(tags[i])

  
  for ind,y in enumerate(p):
    tag=indeksi_u_tagove[y]
    p[ind]=tag

  return y_true,p

def padding(X,padding):
  #Funkcija koja dodaje ekstra tokene prekratkim rečenicama. To je potrebno jer za ulaz u NN trebamo rečenice jednakih duljina.

  for i,x in enumerate(X):
    padding_duljina=50-len(x)
    
    x[len(x):]=[padding for i in range(padding_duljina)]
    #x.append(X.index[i])




#Tu se kreira dataset te filtriraju stupci
tekst=kreiraj_dataset()
data=tekst[["Rečenica","Riječ","NER_tag"]]



#Liste sadrže sve riječi i tagove u korpusu
words = list(set(data["Riječ"].values))
words.append("ENDPAD")
n_words = len(words)
tags = list(set(data["NER_tag"].values))
n_tags = len(tags)



#Riječnici koji pretvaraju stringove u brojeve
max_len = 50
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
# tokeniziram riječi i NER_tagove



#tokeni ovdje ustvari znače numerički token. Dakle ovo su stringovi pretvoreni u brojeve.
word_tokeni=[]
NER_tokeni=[]

for word in data["Riječ"]:
  word_tokeni.append(word2idx[word])

for tag in data["NER_tag"]:
  NER_tokeni.append(tag2idx[tag])

data["Riječ_tok"]=word_tokeni
data["NER_tok"]=NER_tokeni



#Ovdje skupljam riječi iz rečenica u liste. Svaka rečenica je jedna lista. 
X_za_embedding=data.groupby("Rečenica")["Riječ"].apply(list)


#tu trazim indekse recenica koje imaju vise od 75 rijeci jer se kasnije koristi input od 75 riječi za model. 
x_izbaciti=[]
for i,x in enumerate(X_za_embedding):
  if len(x)>=max_len:
    
    x_izbaciti.append(i+1)


#Izbacujem rečenice s više od 75 riječi
X_za_embedding=X_za_embedding.drop(index=x_izbaciti)


#Isto kao i za X uz dodadno paddanje Y listi.
Y=data.groupby("Rečenica")["NER_tok"].apply(list)

y_izbaciti=[i-1 for i in x_izbaciti]
Y=Y.drop(index=y_izbaciti)
Y = pad_sequences(maxlen=max_len, sequences=Y, padding="post", value=tag2idx["O"])

  
padding(X_za_embedding,"ENDPAD")

X_za_embedding=X_za_embedding.reset_index(drop=True)



#Tu loadam embeddan korpus koji ide u  model
with open('elmo.pkl', 'rb') as f:
    ELMO = pickle.load(f)


    
batch_size=32

#train i test skup

indeksi=list(X_za_embedding.index)

X_tr, X_te, y_tr, y_te,indeksi_tr,indeksi_te = train_test_split(ELMO, Y, indeksi, test_size=0.1, random_state=1000)
X_tr, X_val = X_tr[:1213*batch_size], X_tr[-135*batch_size:]

y_tr, y_val = y_tr[:1213*batch_size], y_tr[-135*batch_size:]
y_tr = np.array(y_tr).reshape(np.array(y_tr).shape[0], np.array(y_tr).shape[1], 1)
y_val = np.array(y_val).reshape(np.array(y_val).shape[0], np.array(y_val).shape[1], 1)



#MODEL

input_text = Input(shape=(max_len,1024),dtype=tf.float32)
x = Bidirectional(LSTM(units=512, return_sequences=True,
                        dropout=0.2,recurrent_dropout=0.2))(input_text) 
x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           dropout=0.2,recurrent_dropout=0.2))(x)
x = add([x, x_rnn])
out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)

model = Model(input_text, out)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

history = model.fit(np.array(X_tr), np.array(y_tr), validation_data=(np.array(X_val), np.array(y_val)),
                    batch_size=batch_size, epochs=3, verbose=1)




#računanje predviđanja za sve rečenice iz test skupa te spremanje u listu
indeksi_u_tagove={list(tag2idx.values())[k]:str.upper(list(tag2idx.keys())[k]) for k in range(0,11)}

y_true=[]
y_pred=[]
for i in range(len(y_te)):
  y1,y2=predvidi_jedan(i)
  y_true.append(y1)
  y_pred.append(y2)


#bitne metrike koje govore o kvaliteti modela
cl_report=classification_report(y_true, y_pred)
for i in cl_report.split("\n"):
  print(i)

#f1 score je ispod 0.76, ja sam max dobio 0.79

C:\Users\Ivan Emanuel Pavlov\Anaconda3\lib\site-packages\ipykernel_launcher.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Ivan Emanuel Pavlov\Anaconda3\lib\site-packages\ipykernel_launcher.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Train on 21704 samples, validate on 4320 samples
Epoch 1/3
21704/21704 [==============================] - 284s 13ms/step - loss: 0.0637 - accuracy: 0.9852 - val_loss: 0.0473 - val_accuracy: 0.9896
Epoch 2/3
21704/21704 [==============================] - 269s 12ms/step - loss: 0.0433 - accuracy: 0.9904 - val_loss: 0.0430 - val_accuracy: 0.9910
Epoch 3/3
21704/21704 [==============================] - 269s 12ms/step - loss: 0.0391 - accuracy: 0.9916 - val_loss: 0.0356 - val_accuracy: 0.9925
              precision    recall  f1-score   support

         DER       0.68      0.72      0.70        32
         LOC       0.90      0.87      0.88       644
         MIS       0.55      0.40      0.46       332
         ORG       0.63      0.76      0.69       563
         PER       0.87      0.82      0.84       660

   micro avg       0.76      0.75      0.76      2231
   macro avg       0.72      0.71      0.71      2231
weighted avg       0.76      0.75      0.76      2231



Train on 21704 samples, validate on 4320 samples
Epoch 1/2
21704/21704 [==============================] - 301s 14ms/step - loss: 0.0352 - accuracy: 0.9926 - val_loss: 0.0352 - val_accuracy: 0.9933
Epoch 2/2
21704/21704 [==============================] - 272s 13ms/step - loss: 0.0314 - accuracy: 0.9936 - val_loss: 0.0277 - val_accuracy: 0.9948


              precision    recall  f1-score   support

         DER       0.83      0.75      0.79        32
         LOC       0.86      0.90      0.88       644
         MIS       0.58      0.53      0.56       332
         ORG       0.74      0.69      0.71       563
         PER       0.86      0.86      0.86       660

   micro avg       0.79      0.78      0.78      2231
   macro avg       0.77      0.75      0.76      2231
weighted avg       0.79      0.78      0.78      2231



# Predict rečenice iz test skupa

In [214]:
# biramo neku rečenicu te predictamo NER_tag vjerojatnost(znači ima ih 11, pa 11 rezultata) za neki token
# s argmax biramo najvjerojatniji tag od tih 11, ovaj axis označava da biramo iz zadnje dimenzije arraya. Na kraju je 50 tagova, za 50 tokena (zbog paddinga)

# s i biramo neku rečenicu, treba odabrati neki i koji ima u test setu


i = 54
p = model.predict(np.array(X_te[i:i+batch_size]))[0]
p = np.argmax(p, axis=-1)
print("{:15} {:5}: {}".format("Word", "True","Pred"))
print("="*30)
for w, true, pred in zip(X_za_embedding[indeksi_te[i]], y_te[i], p):
    if w != "ENDPAD":
        print("{:15}:{:5} {}".format(w, tags[true],tags[pred]))

Word            True : Pred
O              :O     O
životu         :O     O
u              :O     O
Hrvatskoj      :B-LOC B-LOC
,              :O     O
o              :O     O
životu         :O     O
u              :O     O
Malaviju       :B-LOC B-LOC
,              :O     O
o              :O     O
životu         :O     O
u              :O     O
Cape           :B-LOC B-LOC
Maclearu       :I-LOC I-LOC
.              :O     O


# Predictanje novog teksta

In [216]:
#ovdje upisati svoj path gdje se nalazi elmo HR model
path= r"C:\Users\Ivan Emanuel Pavlov\OneDrive - Prirodoslovno-matematički fakultet\Megatrend_zadatak\ELMO-croatian"

elmo_model = Embedder(path,batch_size=32)


2021-12-03 08:31:40,000 WARNING: Could not find config.  Trying C:\Users\Ivan Emanuel Pavlov\OneDrive - Prirodoslovno-matematički fakultet\Megatrend_zadatak\ELMO-croatian\cnn_50_100_512_4096_sample.json
2021-12-03 08:31:40,001 WARNING: Could not find config.  Trying C:\Users\Ivan Emanuel Pavlov\Anaconda3\lib\site-packages\elmoformanylangs\configs\cnn_50_100_512_4096_sample.json
2021-12-03 08:31:40,014 INFO: char embedding size: 2622
2021-12-03 08:31:41,559 INFO: word embedding size: 299927
2021-12-03 08:31:47,086 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(299927, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(2622, 50, padding_idx=2619)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 12

In [217]:


#tu se mogu umetnuti nove tokenizirane (već tokenizirane radi jednostavnosti, inače bi kao input naravno išle nove rečenice koja bi se potom tokenizirale kao ranije u kodu)
#U listi moraju biti minimalno 2 rečenice

test_sentence = [["Živim","u","Mozambiku","te","radim","u","Microsoftu","a","ime","mi","je","Niko","Takkoki"],["Utah","Jazz","je","stacionirana","u","Salt","Lake","City-u","a","najpoznatiji","igrač","im","je","Rudy","Gobert"]]
    
for rec in test_sentence:
    for i in range(max_len-len(rec)):
        rec.append("ENDPAD")

test_sentence_elmo=elmo_model.sents2elmo(np.array(test_sentence),output_layer=-1)



#indeks recenice i
i=1

p = model.predict(np.array(test_sentence_elmo))
p = np.argmax(p,axis=-1)


print("{:15} {}".format("Word","Pred"))
print("="*30)
for w, pred in zip(test_sentence[i], p[i]):
    if w != "ENDPAD":
        #print(pred)
        print("{:15} {}".format(w,tags[pred]))
        




2021-12-03 08:31:48,026 INFO: 1 batches, avg len: 52.0


Word            Pred
Utah            B-ORG
Jazz            I-ORG
je              O
stacionirana    O
u               O
Salt            B-LOC
Lake            I-LOC
City-u          I-LOC
a               O
najpoznatiji    O
igrač           O
im              O
je              O
Rudy            B-PER
Gobert          I-PER


# Izračun ELMO embeddinga za cijeli korpus

In [ ]:
#Kod za izračun ELMO embeddinga za cijeli korpus


#ovdje upisati svoj path gdje se nalazi elmo HR model
# ELMO model se skida s https://github.com/HIT-SCIR/ELMoForManyLangs

path= r"C:\Users\Ivan Emanuel Pavlov\OneDrive - Prirodoslovno-matematički fakultet\Megatrend_zadatak\ELMO-croatian"


#tu se učitava model koji je prethodno skinut s neta te se s njegovom funkcijom sents2elmo izračunavaju embeddingsi za sve rečenice u HR korpusu
elmo_model = Embedder(path,batch_size=32)

em=elmo_model.sents2elmo(np.array(list(X_za_embedding)),output_layer=-1)


with open('elmo_2.pkl', 'wb') as f:
    pickle.dump(em, f)
